In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importing Libraries

In [2]:
import re, os, sys, nltk
import xml.etree.cElementTree as et
import pickle, base64, time
from heapq import *
import math, operator, json

In [3]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("/content/drive/My Drive/Wiki-Search/stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1
#print(stop_words)

In [4]:
pattern = re.compile("[^a-zA-Z0-9]")
cssExp = re.compile(r'{\|(.*?)\|}',re.DOTALL)
linkExp = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)

##XML Parser for reading XML files

In [5]:
# arguments = sys.argv
#wikipedia_dump = sys.argv[1]
index_path = "/content/drive/My Drive/Wiki-Search/"
wikipedia_dump = "/content/drive/My Drive/Wiki-Search/Input-Data/input.xml"
content = et.iterparse(wikipedia_dump, events=("start", "end"))
content = iter(content)
# document_title = open("index/document_title.pickle", "wb")

In [6]:
title_inverted_index = {}
body_inverted_index = {}
category_inverted_index = {}
infobox_inverted_index = {}

In [7]:
fcount, per_page_document = 0, 35000

In [8]:
document_no = 1
title_freq = {}
body_freq = {}
category_freq = {}
infobox_freq = {}
document_title = open("/content/drive/My Drive/Wiki-Search/titles.txt","w+")
document_title_position = []
document_word = {}
start = time.time()
store_stemmed_word = {}

In [9]:
def write_into_file(filename,inverted_object,flag):
    global document_word
    fileptr = open(filename, "w+")
    pointer = 0
    for word in inverted_object:
        posting_list = ",".join(inverted_object[word])
        posting_list = posting_list + "\n"
        if word not in document_word:
            document_word[word] = {}
        document_word[word][flag] = pointer
        fileptr.write(posting_list)
        pointer += len(posting_list)
    fileptr.close()

def write_pickle_file(filename, pickleobj):
    file = open(filename, "wb")
    pickle.dump(pickleobj, file)
    file.close()


##Preprocessing(Tokenization,Casefolding,Stopword removal,Stemming)

In [10]:
for event,context in content:
    tag = re.sub(r"{.*}", "", context.tag)
    
    if event == "end":
        
        if tag == "title":
            
            title_text = context.text
            position = document_title.tell()
            document_title_position.append(position)
            document_title.write(title_text + "\n")
            title_text = title_text.lower()
            try:
                words = re.split(pattern, title_text)
                for word in words:
                    word = word.lower()
                    if word in store_stemmed_word:
                        word = store_stemmed_word[word]
                    else:
                        stem = stemmer.stem(word)
                        store_stemmed_word[word] = stem
                        word = stem
                    if len(word) <= 2:
                        continue
                    if word and word not in stop_words:
                        if word not in title_freq:
                            title_freq[word] = 1
                        else:
                            title_freq[word] += 1
            except:
                pass
        
        elif tag == "text":
            
            body_text = context.text
            body_text = linkExp.sub('',str(body_text))
            body_text = cssExp.sub('',str(body_text))
            try:
                category_words = re.findall("\[\[Category:(.*?)\]\]", body_text);
                if category_words != "":
                    for category_word in category_words:
                        words = re.split(pattern, category_word)
                        for word in words:
                            word = word.lower()
                            if word in store_stemmed_word:
                                word = store_stemmed_word[word]
                            else:
                                stem = stemmer.stem(word)
                                store_stemmed_word[word] = stem
                                word = stem
                            if len(word) <= 2:
                                continue
                            if  word and word not in stop_words:
                                if word not in category_freq:
                                    category_freq[word] = 1
                                else:
                                    category_freq[word] += 1
            except:
                pass
            
            try:

                info_words = re.findall("{{Infobox((.|\n)*?)}}", body_text)
                if info_words != "":
                    for info_word in info_words:
                        for i_word in info_word:
                            words = re.split(pattern, i_word)
                            for word in words:
                                word = word.lower()
                                if word in store_stemmed_word:
                                    word = store_stemmed_word[word]
                                else:
                                    stem = stemmer.stem(word)
                                    store_stemmed_word[word] = stem
                                    word = stem
                                if len(word) <= 2:
                                    continue
                                if word and word not in stop_words:
                                    if word not in infobox_freq:
                                        infobox_freq[word] = 1
                                    else:
                                        infobox_freq[word] += 1
            except:
                pass
                                    
            try:
                words = re.split(pattern, body_text)

                for word in words:
                    word = word.lower()
                    if word in store_stemmed_word:
                        word = store_stemmed_word[word]
                    else:
                        stem = stemmer.stem(word)
                        store_stemmed_word[word] = stem
                        word = stem
                    if len(word) <= 2:
                        continue
                    if word and word not in stop_words:
                        if word not in body_freq:
                            body_freq[word] = 1
                        else:
                            body_freq[word] += 1
            except:
                pass
            
        elif tag == "page":
            d_no = str(document_no)
            for word in body_freq:
                if word not in body_inverted_index:
                    body_inverted_index[word]=[]
                body_inverted_index[word].append(d_no + ":" + str(body_freq[word]))
            
            body_freq.clear()

            for word in title_freq:
                if word not in title_inverted_index:
                    title_inverted_index[word]=[]
                title_inverted_index[word].append(d_no + ":" + str(title_freq[word]))
            
            title_freq.clear()

            for word in category_freq:
                if word not in category_inverted_index:
                    category_inverted_index[word]=[]
                category_inverted_index[word].append(d_no + ":" + str(category_freq[word]))
            
            category_freq.clear()
            
            for word in infobox_freq:
                if word not in infobox_inverted_index:
                    infobox_inverted_index[word]=[]
                infobox_inverted_index[word].append(d_no + ":" + str(infobox_freq[word]))
                
            infobox_freq.clear()
            
            if document_no%80000:
                store_stemmed_word = {}
            
            if document_no%per_page_document==0:
                filename = "/content/drive/My Drive/Wiki-Search/title_" + str(fcount) + ".txt"
                write_into_file(filename,title_inverted_index,'t')
                title_inverted_index.clear()
                
                filename = "/content/drive/My Drive/Wiki-Search/category_" + str(fcount) + ".txt"
                write_into_file(filename,category_inverted_index,'c')
                category_inverted_index.clear()
                
                filename = "/content/drive/My Drive/Wiki-Search/infobox_" + str(fcount) + ".txt"
                write_into_file(filename,infobox_inverted_index,'i')
                infobox_inverted_index.clear()
                
                filename = "/content/drive/My Drive/Wiki-Search/body_text_" + str(fcount) + ".txt"
                write_into_file(filename,body_inverted_index,'b')
                body_inverted_index.clear()
                fcount = fcount + 1
            document_no += 1

In [11]:
print(len(title_inverted_index))
print(len(body_inverted_index))
print(len(category_inverted_index))
print(len(infobox_inverted_index))
print(len(infobox_freq))

23304
227389
11505
21767
0


In [12]:
document_title.close()

##Save output files

In [13]:
filename = "/content/drive/My Drive/Wiki-Search/title_position.pickle"
write_pickle_file(filename,document_title_position)
document_title_position.clear()

In [14]:
filename = "/content/drive/My Drive/Wiki-Search/title_" + str(fcount) + ".txt"
write_into_file(filename,title_inverted_index,'t')
title_inverted_index.clear()

In [15]:
filename = "/content/drive/My Drive/Wiki-Search/category_" + str(fcount) + ".txt"
write_into_file(filename,category_inverted_index,'c')
category_inverted_index.clear()

In [16]:
filename = "/content/drive/My Drive/Wiki-Search/infobox_" + str(fcount) + ".txt"
write_into_file(filename,infobox_inverted_index,'i')
infobox_inverted_index.clear()

In [17]:
filename = "/content/drive/My Drive/Wiki-Search/body_text_" + str(fcount) + ".txt"
write_into_file(filename,body_inverted_index,'b')
body_inverted_index.clear()

In [18]:
fcount = fcount + 1
type_of_files = ["/content/drive/My Drive/Wiki-Search/title_","/content/drive/My Drive/Wiki-Search/category_","/content/drive/My Drive/Wiki-Search/infobox_","/content/drive/My Drive/Wiki-Search/body_text_"]
mapping = {}
mapping["/content/drive/My Drive/Wiki-Search/infobox_"] = 'i'
mapping["/content/drive/My Drive/Wiki-Search/body_text_"] = 'd'
mapping["/content/drive/My Drive/Wiki-Search/title_"] = 't'
mapping["/content/drive/My Drive/Wiki-Search/category_"] = 'c'
output_files = []

##Inverted Indexing/Posting List

In [19]:
for f in type_of_files:
    heap = []
    input_files = []
    output_file = f + "output.txt"
    output_ptr = open(output_file,'w+')
    output_files.append(output_ptr)
    output_f_no = len(output_files) - 1
    
    for file_no in range(fcount):
        fname = f + str(file_no) + ".txt"
        fptr = open(fname,"r")
        if os.stat(fname).st_size == 0:
            continue
        input_files.append(fptr)
    
    if len(input_files) == 0:
        break
    
    for file_no in range(fcount):
        try:
            line = input_files[file_no].readline()[:-1]
#             print(line)
            heap.append((line,file_no))
        except Exception as e:
            print(e)
    
    heapify(heap)
    
    file_no = 0
    try:
        while file_no < fcount:
            line, f_no = heappop(heap)
            word = line.split("-")[0]
            posting_list = line.split("-")[1]
            next_line = input_files[f_no].readline()[: -1]
            if next_line!= "":
                heappush(heap, (next_line, f_no))
            else:
                file_no = file_no + 1

            while file_no < fcount:
                try:
                    next_line, next_f_no = heappop(heap)
                    next_word = next_line.split("-")[0]
                    next_posting_list = next_line.split("-")[1]
                except IndexError:
                    break

                if next_word == word:
                    posting_list = posting_list + "," + next_posting_list
                    next_next_line = input_files[next_f_no].readline()[:-1]
                    if next_next_line:
                        heappush(heap, (next_next_line, next_f_no))
                    else:
                        file_no = file_no + 1
                else:
                    heappush(heap, (next_line, next_f_no))
                    break

            if word not in document_word:
                document_word[word]={}

            document_word[word][mapping[f]] = output_files[output_f_no].tell()
            final_posting_list = posting_list.split(",")

            idf = math.log10(document_no / len(final_posting_list))
            documents_score = {}
            for post in final_posting_list:

                doc_no = int(post.split(":")[0])
                word_freq = int(post.split(":")[1])
                tf = 1 + math.log10(word_freq)
                documents_score[doc_no] = round(idf * tf, 2)

            score = ""
            count = 0
            for document in documents_score.keys():
                score = score + str(document) + ":" + str(documents_score[document]) + ","
                count = count + 1
            score = score[:-1] + "\n"
    #             print(score)
            output_files[output_f_no].write(score)
    except IndexError:
        pass
    
    output_files[output_f_no].close()
    
    for f_no in range(fcount):
        file_ = f + str(f_no) + ".txt"
        os.remove(file_)

In [20]:
filename = "/content/drive/My Drive/Wiki-Search/word_position.json"
with open(filename,"w+") as f:
    json.dump(document_word,f)

In [21]:
end = time.time()
print("Time Taken :- ",(end-start))

Time Taken :-  80.38945889472961
